# Safe Beauty

> In 2005, California implemented the California Safe Cosmetics Program (CSCP) law, which requires companies to report potentially harmful ingredients. The CSCP database has over 370,000 records with detailed information. </br>
> This report focuses on companies that eliminate harmful ingredients from their products and provides a tool to check if your favorite brand is part of this effort to achieve Safe Beauty.

####Is there a correlation between a cosmetic company's brand value or market presence and its ability to adjust its products and comply with CSCP regulations regarding the elimination of ingredients harmful to human health?

To perform the analysis, I will use the official database published by the state of California on the exclusive program website. It can be downloaded from here > https://www.cdph.ca.gov/Programs/CCDPHP/DEODC/OHB/CSCP/Pages/CSCP.aspx


In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Read original CSV and make a copy for work
origin = pd.read_csv('CDPH_Search_results.csv')
df = origin.copy()
df.head(3)

,Product Id,Company,Brand,Product Name,Variant,Product Discontinued Date,Product Submitted Date,Ingredient Name,Function,Unit of Measure,Concentration,Ingredient Submitted Date,Ingredient Removed Date,UPC,Body Area,Product Category,Product Form,Intended Market
0,43485,"Anastasia Beverly Hills, LLC",Anastasia Beverly HIlls,Perfect Brow Pencil (Granite shade_,NaN,NaN,7/15/2016,Titanium dioxide (CI 77891) 13463-67-7 / 1317-...,NaN,mg/g,46.81,7/15/2016,NaN,Not Available,Other (Specify):,Eyeliner/Eyebrow Pencils,Stick or Pencil,Not Available
1,18358,Nail Alliance - Entity,Entity Nudite,Cool Pink Nail Sculpting Powder,NaN,NaN,6/24/2019,Titanium dioxide (CI 77891) 13463-67-7 / 1317-...,NaN,mg/g,5,6/24/2019,NaN,Not Available,Nails,Artificial Nails and Related Products,Powder,Not Available
2,23202,GAP INC.,Gap Outlet,"Light Pink, lip gloss (Lip trio)",Light Pink,1/1/2018,11/7/2014,Titanium dioxide (CI 77891) 13463-67-7 / 1317-...,NaN,NaN,NaN,11/7/2014,NaN,Not Available,Lips,Lip Gloss/Shine,Liquid,Not Available


In [4]:
# Knowing the data set

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390280 entries, 0 to 390279
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   Product Id                 390280 non-null  int64 
 1   Company                    390280 non-null  object
 2   Brand                      390210 non-null  object
 3   Product Name               390280 non-null  object
 4   Variant                    156566 non-null  object
 5   Product Discontinued Date  26354 non-null   object
 6   Product Submitted Date     390280 non-null  object
 7   Ingredient Name            390280 non-null  object
 8   Function                   269986 non-null  object
 9   Unit of Measure            177365 non-null  object
 10  Concentration              178785 non-null  object
 11  Ingredient Submitted Date  390280 non-null  object
 12  Ingredient Removed Date    4897 non-null    object
 13  UPC                        390167 non-null  

## Cleanup of NaN values

There are multiples NaN values but, for this report proposals, I only edit the NaN values in "Brand" and "Concentration" columns.

In [5]:
# 'Brand' NaN values
df['Brand'].isna().sum() # Total = 70
# Fill NaN Brand values with text 'Generic'
df['Brand'].fillna('Generic', inplace=True)

# 'Concentration' NaN values
df['Concentration'].isna().sum() # Total = 203402
# Fill NaN with 0 so as not to lose the entire entry.
df['Concentration'].fillna(0, inplace=True)

# Check
df.loc[(df['Brand'] == 'Generic') | df['Concentration'] == 0, :].sample(3)


,Product Id,Company,Brand,Product Name,Variant,Product Discontinued Date,Product Submitted Date,Ingredient Name,Function,Unit of Measure,Concentration,Ingredient Submitted Date,Ingredient Removed Date,UPC,Body Area,Product Category,Product Form,Intended Market
368093,119355,"Conopco, Inc.",TRESemme,Tresemme Extra Hold Hair Gel,NaN,NaN,12/9/2022,GERANIOL,Fragrance,NaN,0,12/9/2022,NaN,022400624273,Hair or Scalp,Hair Styling Products,Cream/Gel/Foam,General population
54304,27900,Arcadia Beauty Labs LLC,Bitzy,Bitzy Dusk & Starlight Cosmetic Palette,NaN,NaN,11/6/2017,Titanium dioxide (CI 77891) 13463-67-7 / 1317-...,NaN,NaN,0,11/6/2017,NaN,Not Available,Face,Face Powders,Powder,Not Available
30282,116607,Bath & Body Works,Bath & Body Works,Bath & Body Works Men's Collection 24 HR Moist...,NaN,NaN,9/28/2022,CITRONELLOL,Fragrance,NaN,0,9/28/2022,NaN,667557239224,Body (general),Skin Moisturizers (making a cosmetic claim),Cream/Gel/Foam,General population


# Clean wrong data

Some companies did not report ingredient concentration because it considers 'Trade Secret'. I need this value in number, that's why this value will edit with '-1'

In [21]:
# Cuantos registros 'Trade Secret' hay en el dataset?
trade_secret = df['Concentration'] == 'Trade Secret' # 60288

# Reemplazamos los registros 'Trade Secret' por -1
df['Concentration'].replace('Trade Secret', -1, inplace=True)

# Finalmente convertimos la columna Concentration a tipo de dato numerico (float)
df['Concentration'] = pd.to_numeric(df['Concentration'])

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 387595 entries, 0 to 390279
Data columns (total 20 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Product Id                 387595 non-null  int64  
 1   Product Name               387595 non-null  object 
 2   Variant                    154091 non-null  object 
 3   Discontinued               387595 non-null  bool   
 4   Brand                      387595 non-null  object 
 5   Company                    387595 non-null  object 
 6   Ingredient Name            387595 non-null  object 
 7   Function                   269986 non-null  object 
 8   Unit of Measure            176478 non-null  object 
 9   Concentration              387595 non-null  float64
 10  Ingredient Removed         387595 non-null  bool   
 11  Ingredient Submitted Date  387595 non-null  object 
 12  Ingredient Removed Date    4892 non-null    object 
 13  UPC                        38

# Duplicate values

In [23]:
# Check duplicates
df[df.duplicated(keep='first')].sort_values(by=['Product Name', 'Ingredient Name'])

# Drop duplicates
df = df.drop_duplicates()

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 387595 entries, 0 to 390279
Data columns (total 20 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Product Id                 387595 non-null  int64  
 1   Product Name               387595 non-null  object 
 2   Variant                    154091 non-null  object 
 3   Discontinued               387595 non-null  bool   
 4   Brand                      387595 non-null  object 
 5   Company                    387595 non-null  object 
 6   Ingredient Name            387595 non-null  object 
 7   Function                   269986 non-null  object 
 8   Unit of Measure            176478 non-null  object 
 9   Concentration              387595 non-null  float64
 10  Ingredient Removed         387595 non-null  bool   
 11  Ingredient Submitted Date  387595 non-null  object 
 12  Ingredient Removed Date    4892 non-null    object 
 13  UPC                        38

# Orden & Organization

To simplify the information regarding the ingredients that have already been removed from the formulas and the products that have been discontinued from the sale, I will create two new boolean columns. These columns will return true or false depending on whether the ingredient has been removed or the product has been removed from sale. This will allow for easy filtering and analysis of the data.

In [24]:
# Was the ingredient removed? True/False
df['Ingredient Removed'] = df['Ingredient Removed Date'].notnull()

# Was the product discontinued from the sale? True/False
df['Discontinued'] = df['Product Discontinued Date'].notnull()

# Reordering columns for best analysis
df = df.reindex(columns=[
    'Product Id', 'Product Name', 'Variant', 'Discontinued', 'Brand', 'Company', 
    'Ingredient Name', 'Function', 'Unit of Measure', 'Concentration',
    'Ingredient Removed', 'Ingredient Submitted Date', 'Ingredient Removed Date', 'UPC',
    'Product Submitted Date', 'Product Discontinued Date', 
    'Product Category', 'Product Form', 'Body Area', 'Intended Market'
])

# Check
df.sample(2)

,Product Id,Product Name,Variant,Discontinued,Brand,Company,Ingredient Name,Function,Unit of Measure,Concentration,Ingredient Removed,Ingredient Submitted Date,Ingredient Removed Date,UPC,Product Submitted Date,Product Discontinued Date,Product Category,Product Form,Body Area,Intended Market
32277,115507,Bath & Body Works Men's Collection Cologne Bou...,NaN,False,Bath & Body Works,Bath & Body Works,Hydroxycitronellal 107-75-5,Fragrance,NaN,0.000000,False,9/1/2022,NaN,667557237503,9/1/2022,NaN,Cologne,Spray - Non-Aerosol,Neck/chest/decolletage area,General population
353329,118053,MINT MATTIFYING FACE MIST,NaN,False,The Body Shop International Limited,"Buth-na-Bodhaige, Inc",Limonene 5989-27-5,Fragrance,Percent,0.030492,False,11/1/2022,NaN,5028197889494,11/1/2022,NaN,Skin Fresheners,Liquid,Face,General population


## Format dates in a new dataset

In [25]:
# df copy
df_final = df.copy()

# Apply datetime to the 4 columns with dates
df_final[['Product Discontinued Date', 'Product Submitted Date', 'Ingredient Submitted Date', 'Ingredient Removed Date']
   ] = df_final[['Product Discontinued Date', 'Product Submitted Date', 'Ingredient Submitted Date', 'Ingredient Removed Date']
          ].apply(pd.to_datetime, format="%m/%d/%Y")

'''
The format change caused errors in a total of 117 rows, specifically in the 'Ingredient Removed Date' column: 
in 116 rows, the year 2013 was replaced by 2103, and in one row, 2014 was replaced by 2104. 
To correct this, I use lambda functions and the where method from NumPy. 
'''

wrong_13 = df_final['Ingredient Removed Date'].apply(lambda x: x.replace(year=2013))
wrong_14 = df_final['Ingredient Removed Date'].apply(lambda x: x.replace(year=2014))
df_final['Ingredient Removed Date'] = np.where(df_final['Ingredient Removed Date'].dt.year == 2103, wrong_13, df_final['Ingredient Removed Date'])
df_final['Ingredient Removed Date'] = np.where(df_final['Ingredient Removed Date'].dt.year == 2104, wrong_14, df_final['Ingredient Removed Date'])

# Order & check values
df_final.sort_values(by='Ingredient Removed Date', ascending=False).head(3)


,Product Id,Product Name,Variant,Discontinued,Brand,Company,Ingredient Name,Function,Unit of Measure,Concentration,Ingredient Removed,Ingredient Submitted Date,Ingredient Removed Date,UPC,Product Submitted Date,Product Discontinued Date,Product Category,Product Form,Body Area,Intended Market
300067,91252,Que Bella Purifying Tea Tree Mud Mask,NaN,False,Que Bella,Design Plus Health & Beauty Ltd,LINALOOL,Fragrance,Percent,0.0002,True,2021-12-17,2023-03-09,5060242152085,2021-12-17,NaT,Facial Masks,Cream/Gel/Foam,Face,General population
152442,100204,Hair Biology Thickening Serum,NaN,False,Hair Biology,The Procter & Gamble Company (Cincinnati),Glycol ethers,Fragrance,NaN,0.0000,True,2022-03-08,2023-03-01,037000642251,2022-03-08,NaT,Hair Styling Products,Cream/Gel/Foam,Hair or Scalp,General population
152468,100204,Hair Biology Thickening Serum,NaN,False,Hair Biology,The Procter & Gamble Company (Cincinnati),Glycol ethers,Fragrance,NaN,0.0000,True,2022-03-08,2023-03-01,037000642251,2022-03-08,NaT,Other Hair Care Product,Spray - Non-Aerosol,Hair or Scalp,General population


# And now some questions:


### According to the submitted date: which companies were the first to report ingredients potentially harmful in their products?

In [26]:
first_reported = df_final.sort_values(by='Product Submitted Date')
first_reported_top_companies = first_reported['Company'].unique()[:20]
df_first_reported = pd.DataFrame()
for i in first_reported_top_companies:
  company = first_reported.loc[first_reported['Company'] == i]
  first_data = company['Product Submitted Date'].iloc[0]
  df_first_reported = pd.concat([df_first_reported, pd.DataFrame({'Company': [company['Company'].iloc[0]], 'First Submit': [pd.to_datetime(first_data)], 'Product Name' : company['Product Name'].iloc[0]})], ignore_index=True)

df_first_reported

,Company,First Submit,Product Name
0,New Avon LLC,2009-06-17,ULTRA COLOR RICH EXTRA PLUMP LIPSTICK-ALL SHADES
1,J. Strickland & Co.,2009-07-01,Glover's Medicated Shampoo
2,OPI PRODUCTS INC.,2009-07-13,ABSOLUTE Precision Color Powder System Translu...
3,"GOJO Industries, Inc.",2009-07-23,White Premium Lotion Soap
4,"CHANEL, INC",2009-08-10,Professional Eyeshadow Base
5,"Aloecare International, LLC",2009-08-11,Aloe Up Lil' Kids SPF 45 Aloe Based Sunscreen ...
6,"Entity Beauty, Inc.",2009-08-16,Entity One Nudite Pink UV Gel
7,Revlon Consumer Product Corporation,2009-08-18,Pure Blends Natural Lipgloss-Natural
8,Dermalogica,2009-08-21,Skin Prep Scrub
9,CLARINS S.A.,2009-08-24,ECLAT MINUTE EMBELLISSEUR DE TEINT


## And the last reported?

In [27]:
last_reported = df_final.sort_values(by='Product Submitted Date', ascending=False)
last_reported_top_companies = last_reported['Company'].unique()[:20]
df_last_reported = pd.DataFrame()
for i in last_reported_top_companies:
  company = last_reported.loc[last_reported['Company'] == i]
  last_data = company['Product Submitted Date'].iloc[0]
  df_last_reported = pd.concat([df_last_reported, pd.DataFrame({'Company': [company['Company'].iloc[0]], 'Last Submit': [pd.to_datetime(last_data)], 'Product Name' : company['Product Name'].iloc[0]})], ignore_index=True)

df_last_reported

,Company,Last Submit,Product Name
0,"e.l.f. Cosmetics, Inc.",2023-03-17,Lock On Liner And Brow Cream
1,JAFRA COSMETICS INTERNATIONAL,2023-03-17,Sun spray on sunscreen broad spectrum SPF 50
2,Art of Beauty,2023-03-16,Stimulating Citrus Massage Oil
3,Belcam Inc.,2023-03-16,G'eau Blue_ EDT
4,Arcadia Beauty Labs LLC,2023-03-16,Strawberry Leopard Semi-Permanent Conditioning...
5,"Well People, Inc.",2023-03-16,Hydrolip Gloss
6,Juice Beauty,2023-03-16,GREEN APPLE Firming Body Moisturizer
7,"Ruichang Ruimei Cosmetics Co., Ltd",2023-03-16,DTB TW RAINBOW GLOSS WAND .3Z
8,"Yanbal USA, Inc",2023-03-15,ICONO DE YANBAL EAU DE PARFUM
9,Henkel Corporation,2023-03-15,Weekend Hair Dry Shampoo


## How many companies and brands reported potentially harmful ingredients?

In [28]:
total_companies = df_final['Company'].nunique()
total_brands = df_final['Brand'].nunique()
print(f'The database includes {total_brands} brands of {total_companies} companies.') 

The database includes 3078 brands of 846 companies.


## How many products unique were reported?

In [29]:
total_products = df_final['Product Id'].nunique()
print(f'The database contains {total_products} unique products') 

The database contains 119221 unique products


## How many ingredients unique were reported

In [30]:
unique_ingredients = df_final['Ingredient Name'].nunique()
print(f'The database contains {unique_ingredients} unique ingredients') 

The database contains 323 unique ingredients


## What are the 20 companies that reported the largest number of products?

In [31]:
products_by_company = df.groupby('Company')['Product Name'].nunique().sort_values(ascending=False)
products_by_company.head(20)

Company
Bath & Body Works                            2365
American International Industries            1878
Nail Alliance - Entity                       1869
L'Oreal USA                                  1768
The Procter & Gamble Company (Cincinnati)    1462
Revlon Consumer Product Corporation          1409
AMOREPACIFIC US INC.                         1377
Buth-na-Bodhaige, Inc                        1316
Victoria's Secret Beauty                     1314
Conopco, Inc.                                1152
Coty                                          939
Hand & Nail Harmony, Inc                      876
Anastasia Beverly Hills, LLC                  861
S+                                            758
Lush Ltd                                      722
Arcadia Beauty Labs LLC                       684
New Avon LLC                                  671
Too Faced Cosmetics                           569
LABORATOIRES M&L                              568
Lush Manufacturing Ltd.                   

## What are the 20 companies that reported the largest number of ingredients?

In [32]:
ingredients_by_company = df.groupby(['Company'])['Ingredient Name'].nunique().sort_values(ascending=False)
ingredients_by_company.head(20)

Company
AMOREPACIFIC US INC.                         56
Universal Beauty Products, Inc.              54
The Procter & Gamble Company (Cincinnati)    54
Charlotte Tilbury Beauty Ltd                 48
Miniso Depot CA Inc                          48
L'Oreal USA                                  48
Victoria's Secret Beauty                     47
Henkel Corporation                           46
Parfums de Coeur, Ltd., d/b/a PDC Brands     46
Kao USA Inc.                                 46
LE PARFUMEUR SAS                             46
GUERLAIN                                     45
Johnson & Johnson Consumer, Inc              45
Molton Brown Ltd                             45
COLGATE-PALMOLIVE COMPANY                    45
Arcadia Beauty Labs LLC                      43
THG Beauty Limited                           42
United Natural Foods, Inc                    42
Bath & Body Works                            41
Conopco, Inc.                                41
Name: Ingredient Name, dtype: in

## What are the top 5 reported ingredients?

In [33]:
df_final.groupby(['Ingredient Name'])['Ingredient Name'].size().sort_values(ascending=False).head(5)

Ingredient Name
Titanium dioxide (CI 77891) 13463-67-7 / 1317-70-0 / 1317-80-2 / 98084-96-9    108701
Limonene 5989-27-5                                                              35719
LINALOOL                                                                        24585
Benzyl salicylate 118-58-1                                                      12833
CITRONELLOL                                                                     12075
Name: Ingredient Name, dtype: int64

## Reported ingredients vs. Removed Ingredients

In [34]:
total_reported_products = df_final['Product Name'].nunique()
clean_products = df_final.loc[df_final['Ingredient Removed'] == True]
total_reported_products
total_removed_ingredients = clean_products['Product Name'].nunique()
print(f'There are {total_reported_products} reported products. In {total_removed_ingredients} products, have already been the ingredient removed')

There are 56694 reported products. In 1535 products, have already been the ingredient removed


## What products are already cleaned for potentially harmful ingredients?

In [35]:
clean_products = df_final.loc[df_final['Ingredient Removed'] == True]
clean_products.sample(2)

,Product Id,Product Name,Variant,Discontinued,Brand,Company,Ingredient Name,Function,Unit of Measure,Concentration,Ingredient Removed,Ingredient Submitted Date,Ingredient Removed Date,UPC,Product Submitted Date,Product Discontinued Date,Product Category,Product Form,Body Area,Intended Market
343915,34864,TITANIUM DIOXIDE,WHITE,False,SUN CHEMICAL,"NINGBO MEITELI COSMETICS CO.,LTD.",Isopropyl alcohol manufacture using strong acids,NaN,mg/g,4.50,True,2018-10-31,2018-11-01,Not Available,2018-10-31,NaT,Lip Balm (making a cosmetic claim),Powder,Nails,Not Available
71147,95893,Cantu Shea Butter for Natural Hair Curl Stretc...,NaN,False,Cantu,"Parfums de Coeur, Ltd., d/b/a PDC Brands",Isopropyl Alcohol,Other (Specify),Percent,0.46,True,2022-01-28,2022-06-22,817513017469,2022-01-28,NaT,Hair Conditioners (rinse-out),Cream/Gel/Foam,Hair or Scalp,General population


## Export database to excel

In [36]:
clean_products.to_excel(r'clean_products.xlsx')